In [1]:
import pandas as pd
from datetime import datetime

In [2]:
path_pop = 'population_communes.csv'
population = pd.read_csv(path_pop,sep=",")

population.rename(columns={'Code département':'Code_departement'}, inplace=True)
population.head()

,Code_departement,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value
0,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population totale,184
1,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population municipale,177
2,55,10.0,2,44,Abainville,1,Grand Est,Population totale,310
3,55,10.0,2,44,Abainville,1,Grand Est,Population municipale,305
4,60,11.0,1,32,Abancourt,1,Hauts-de-France,Population totale,658


In [3]:
path_surface = 'surface_departements.csv'
surface = pd.read_csv(path_surface,sep=",")
surface.head()

,code_insee,nom,surf_km2
0,974,La Réunion,2505.0
1,11,Aude,6343.0
2,43,Haute-Loire,5003.0
3,13,Bouches-du-Rhône,5247.0
4,47,Lot-et-Garonne,5385.0


In [4]:
table1 = population[population['variable'] == 'Population municipale'][['Code_departement','value']].groupby('Code_departement').sum().reset_index()
table1.rename(columns={'value':'valueMunicipale'}, inplace=True)

In [5]:
table2 = population[population['variable'] == 'Population totale'][['Code_departement','value']].groupby('Code_departement').sum().reset_index()
table2.rename(columns={'value':'valueTotale'}, inplace=True)

In [6]:
table1.head()

,Code_departement,valueMunicipale
0,01,638425
1,02,536136
2,03,339384
3,04,162565
4,05,141107


In [7]:
table2.head()

,Code_departement,valueTotale
0,01,655171
1,02,549587
2,03,349336
3,04,167331
4,05,146148


In [8]:
result = table1.merge(table2, on='Code_departement')
result.head()

,Code_departement,valueMunicipale,valueTotale
0,01,638425,655171
1,02,536136,549587
2,03,339384,349336
3,04,162565,167331
4,05,141107,146148


In [9]:
result['part_Muni_Totale'] = round(result['valueMunicipale'] / result['valueTotale'],2) * 100

In [10]:
result.head()

,Code_departement,valueMunicipale,valueTotale,part_Muni_Totale
0,01,638425,655171,97.0
1,02,536136,549587,98.0
2,03,339384,349336,97.0
3,04,162565,167331,97.0
4,05,141107,146148,97.0


In [11]:
table3 = population[population['variable'] == 'Population totale'][['Code région','value']].groupby('Code région').sum().reset_index()
table3.rename(columns={'value':'valueRegion'}, inplace=True)

In [12]:
table3.head()

,Code région,valueRegion
0,1,400170
1,2,382294
2,3,271829
3,4,862814
4,11,12258425


In [13]:
ref = population[['Code_departement','Code région']].drop_duplicates()
ref.head()

,Code_departement,Code région
0,64,75
2,55,44
4,60,32
5,59,32
8,54,44


In [14]:
result2 = ref.merge(result, on='Code_departement').merge(table3, on='Code région')

In [15]:
result2.head()

,Code_departement,Code région,valueMunicipale,valueTotale,part_Muni_Totale,valueRegion
0,64,75,673986,694279,97.0,6092505
1,24,75,414789,426667,97.0,6092505
2,16,75,353288,365697,97.0,6092505
3,33,75,1566679,1595903,98.0,6092505
4,79,75,374743,385495,97.0,6092505


In [16]:
result2['part_Muni_Region'] = round(result2['valueMunicipale'] / result2['valueRegion'],2) * 100

In [17]:
result2.head()

,Code_departement,Code région,valueMunicipale,valueTotale,part_Muni_Totale,valueRegion,part_Muni_Region
0,64,75,673986,694279,97.0,6092505,11.0
1,24,75,414789,426667,97.0,6092505,7.0
2,16,75,353288,365697,97.0,6092505,6.0
3,33,75,1566679,1595903,98.0,6092505,26.0
4,79,75,374743,385495,97.0,6092505,6.0


In [18]:
pv = pd.pivot_table(population[['Code_departement','value','variable']], index=['Code_departement','variable'], aggfunc='sum').reset_index()
pv2 = pv.pivot(index = 'Code_departement', columns = 'variable', values = 'value')
pv3 = pv2.reset_index()
pv3.head()

variable,Code_departement,Population municipale,Population totale
0,01,638425,655171
1,02,536136,549587
2,03,339384,349336
3,04,162565,167331
4,05,141107,146148


In [19]:
t = pv3.merge(ref,on='Code_departement')
t['part_Muni_Totale'] = round(t['Population municipale'] / t['Population totale'],2) * 100
t.head()

,Code_departement,Population municipale,Population totale,Code région,part_Muni_Totale
0,01,638425,655171,84,97.0
1,02,536136,549587,32,98.0
2,03,339384,349336,84,97.0
3,04,162565,167331,93,97.0
4,05,141107,146148,93,97.0


In [22]:
pv4 = pd.pivot_table(population[['Code région','value','variable']], index=['Code région','variable'], aggfunc='sum').reset_index()
pv5 = pv4.pivot(index = 'Code région', columns = 'variable', values = 'value')
pv6 = pv5.reset_index()
pv6.head()

variable,Code région,Population municipale,Population totale
0,1,394110,400170
1,2,376480,382294
2,3,269352,271829
3,4,852924,862814
4,11,12117132,12258425


In [23]:
t2 = t.merge(pv6, on = 'Code région' )
t2.head()

,Code_departement,Population municipale_x,Population totale_x,Code région,part_Muni_Totale,Population municipale_y,Population totale_y
0,01,638425,655171,84,97.0,7916889,8104357
1,03,339384,349336,84,97.0,7916889,8104357
2,07,325157,334591,84,97.0,7916889,8104357
3,15,145969,151615,84,96.0,7916889,8104357
4,26,508006,522276,84,97.0,7916889,8104357


In [25]:
t2['part_Muni_Region'] = round(t2['Population municipale_x'] / t2['Population municipale_y'],2) * 100
t2.head()

,Code_departement,Population municipale_x,Population totale_x,Code région,part_Muni_Totale,Population municipale_y,Population totale_y,part_Muni_Region
0,01,638425,655171,84,97.0,7916889,8104357,8.0
1,03,339384,349336,84,97.0,7916889,8104357,4.0
2,07,325157,334591,84,97.0,7916889,8104357,4.0
3,15,145969,151615,84,96.0,7916889,8104357,2.0
4,26,508006,522276,84,97.0,7916889,8104357,6.0


In [31]:
t3 = t2.merge(surface, left_on = 'Code_departement', right_on='code_insee')
t3.head()

,Code_departement,Population municipale_x,Population totale_x,Code région,part_Muni_Totale,Population municipale_y,Population totale_y,part_Muni_Region,code_insee,nom,surf_km2
0,01,638425,655171,84,97.0,7916889,8104357,8.0,01,Ain,5784.0
1,03,339384,349336,84,97.0,7916889,8104357,4.0,03,Allier,7379.0
2,07,325157,334591,84,97.0,7916889,8104357,4.0,07,Ardèche,5566.0
3,15,145969,151615,84,96.0,7916889,8104357,2.0,15,Cantal,5774.0
4,26,508006,522276,84,97.0,7916889,8104357,6.0,26,Drôme,6558.0


In [32]:
t3['densite'] = round(t3['Population municipale_x'] / t3['surf_km2'],2)
t3.head()

,Code_departement,Population municipale_x,Population totale_x,Code région,part_Muni_Totale,Population municipale_y,Population totale_y,part_Muni_Region,code_insee,nom,surf_km2,densite
0,01,638425,655171,84,97.0,7916889,8104357,8.0,01,Ain,5784.0,110.38
1,03,339384,349336,84,97.0,7916889,8104357,4.0,03,Allier,7379.0,45.99
2,07,325157,334591,84,97.0,7916889,8104357,4.0,07,Ardèche,5566.0,58.42
3,15,145969,151615,84,96.0,7916889,8104357,2.0,15,Cantal,5774.0,25.28
4,26,508006,522276,84,97.0,7916889,8104357,6.0,26,Drôme,6558.0,77.46


In [34]:
t3[t3['code_insee'] == '43']

,Code_departement,Population municipale_x,Population totale_x,Code région,part_Muni_Totale,Population municipale_y,Population totale_y,part_Muni_Region,code_insee,nom,surf_km2,densite
7,43,227339,234613,84,97.0,7916889,8104357,3.0,43,Haute-Loire,5003.0,45.44
